# Security Dataset Generation
This Notebook is intended to generate a security dataset. So far it's columns will consist of paragraph scenarios intended to feed into the LLM we will fine tune

### Prompt Template
Prompt: Generate a security scenario involving a buisness. The scenario should include specific risks or vulnerabilities, such as cloud systems, employee actions, or third-party services. The scenario should also allud to potential security attacks that could happen in the organization.

In [2]:
attack_map = {
    "Phishing": "A social engineering attack to steal sensitive information by impersonating a trustworthy entity.",
    "Ransomware": "Malware that encrypts a victim's files, demanding a ransom to restore access.",
    "Man-in-the-Middle": "An attack where a hacker intercepts communication between two parties to steal data.",
    "Insider Threats": "Security risks originating from within the organization, such as malicious employees.",
    "DDoS": "Distributed denial-of-service attacks to overwhelm systems, causing disruption of services.",
    "SQL-injection": "Exploiting a web app's database query to access or manipulate data",
    "Cross-Site-Scripting": "Injecting malicious scripts into web pages viewed by others.",
    "Privilege Escalation": "This attack exploits vulnerabilities within business systems to gain higher access levels and steal or manipulate data."
}

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import openai
import random
import json
import os

# Set up your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Function to generate a scenario from GPT-4
def generate_scenario():
    prompt = """
    Generate a security scenario involving a buisness. The scenario should include specific risks or vulnerabilities, 
    such as cloud systems, employee actions, or third-party services. The scenario should also allud to potential security 
    attacks that could happen in the organization."""
    response = openai.completions.create(
        model= "gpt-4", 
        prompt=prompt,
        max_tokens=150,  # Adjust based on desired scenario length
        temperature=0.7,  # Adjust creativity
    )
    return response['choices'][0]['text'].strip()

# Main function to generate 1,000 scenarios
def generate_dataset(num_scenarios=1000):
    dataset = []
    
    for i in range(num_scenarios):
        scenario = generate_scenario()
        # attacks = map_attacks_to_scenario(scenario)
        
        entry = {
            "id": i + 1,
            "scenario": scenario,
            # "attacks": [attack_map[attack] for attack in attacks]
        }
        dataset.append(entry)
        
        # Print progress
        if (i + 1) % 100 == 0:
            print(f"Generated {i + 1} scenarios.")
    
    return dataset

c:\Users\evang\anaconda3\envs\LLMenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
df = generate_dataset()

# # Optionally, save to a JSON or CSV file
# with open('generated_dataset.csv', 'w') as f:
#     json.dump(df, f, indent=4)

#  Save as CSV
df.to_csv("output.csv", index=False)

print("Dataset generated and saved!")

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Invalid model-index. Not loading eval results into CardData.
Some weights of BertLMHeadModel were not initialized from the model checkpoint at Intel/dynamic_tinybert and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: Object of type BertLMHeadModel is not JSON serializable

In [ ]:
# Main script to generate 1,000 entries
dataset = []
for i in range(200):
    entry = generate_data_entry()
    dataset.append({"id": i + 1, "data": entry})

# Optionally, save to a JSON or CSV file
with open('generated_dataset.csv', 'w') as f:
    json.dump(dataset, f, indent=4)

print("Dataset generated and saved!")